In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
sentence = 'Repeat is the best medicine for memory'.split()
vocab = list(set(sentence))
print(vocab)

['Repeat', 'best', 'is', 'the', 'medicine', 'memory', 'for']


In [3]:
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}
word2index[''] = 0

In [4]:
print(word2index['memory'])

6


In [5]:
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'Repeat', 2: 'best', 3: 'is', 4: 'the', 5: 'medicine', 6: 'memory', 7: 'for', 0: ''}


In [6]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq

In [7]:
X, Y = build_data(sentence, word2index)

In [8]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first = True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings = vocab_size, embedding_dim = input_size) # Word Embedding
        self.rnn_layer = nn.RNN(input_size, hidden_size, batch_first = batch_first) # 입력 차원, 은닉 상태의 크기 정의
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야 함

    def forward(self, x):
        # 1. Embedding Layer
        # 크기 변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)

        # 2. RNN 층
        # 크기 변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)

        # 3. 최종 출력 층
        # 크기 변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)

        # 4. view를 통해서 배치 차원 제거
        # 크기 변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [9]:
vocab_size = len(word2index) # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. 토큰을 크기에 포함한다
input_size = 5 # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20 # RNN의 은닉층 크기

In [10]:
model = Net(vocab_size, input_size, hidden_size, batch_first = True)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(params = model.parameters())

output = model(X)
print(output)

tensor([[-0.1881,  0.2414, -0.2003,  0.1540,  0.5311, -0.1876,  0.1616, -0.4516],
        [ 0.1173,  0.0650,  0.1111,  0.2593, -0.1003, -0.0709,  0.3022,  0.3288],
        [-0.2729,  0.4813, -0.0122,  0.3854,  0.2483, -0.3025, -0.1151, -0.3288],
        [-0.0434,  0.3140, -0.0732, -0.0345,  0.5671,  0.0079, -0.2296, -0.3975],
        [ 0.3499,  0.2116,  0.3907, -0.1391, -0.0119,  0.2758,  0.0632,  0.4179],
        [-0.2331,  0.2217,  0.0028,  0.4986,  0.2343, -0.5861,  0.2349,  0.2304]],
       grad_fn=<ViewBackward0>)


In [11]:
print(output.shape)

torch.Size([6, 8])


In [12]:
decode = lambda y: [index2word.get(x) for x in y]

In [13]:
for step in range(201):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실 값 계산
    loss = loss_func(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()

    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(' '.join(['Repeat'] + decode(pred)))
        print()

[01/201] 2.0685 
Repeat the for Repeat the for is

[41/201] 1.4013 
Repeat is the best medicine for memory

[81/201] 0.7908 
Repeat is the best medicine for memory

[121/201] 0.4139 
Repeat is the best medicine for memory

[161/201] 0.2309 
Repeat is the best medicine for memory

[201/201] 0.1434 
Repeat is the best medicine for memory

